<a href="https://colab.research.google.com/github/ebokai/google_colab/blob/main/20230324_RBM_TORCH_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nv = 3
nh = 3
n = nv + nh

In [3]:
learning_rate = 0.01
epochs = 25
batch_size = 100
N = 10000
k = 3 # gibbs sampling steps

In [4]:
class RBM(torch.nn.Module):
    
    def __init__(self, nv, nh):
        
        super(RBM, self).__init__()
        
        self.w = torch.nn.Parameter(torch.randn(nv, nh) * 0.01)
        self.a = torch.nn.Parameter(torch.zeros(nv))
        self.b = torch.nn.Parameter(torch.zeros(nh))
        
    def sample_h(self, v):
        
        phv = torch.sigmoid(torch.matmul(v, self.w) + self.b)
        h = torch.bernoulli(phv)
        
        return h, phv
    
    def sample_v(self, h):
        
        pvh = torch.sigmoid(torch.matmul(h, self.w.t()) + self.a)
        v = torch.bernoulli(pvh)
        
        return v, pvh
    
    def forward(self, v):
        h, phv = self.sample_h(v)
        for i in range(k):
            v, pvh = self.sample_v(phv)
            h, phv = self.sample_h(v)
        v, pvh = self.sample_v(phv)
        return v
    
    def free_energy(self, v):
        
        vt = torch.matmul(v, self.a)
        ht = torch.sum(torch.log(1 + torch.exp(torch.matmul(v, self.w) + self.b)), dim = 1)
        
        return -(vt + ht)

In [9]:
def torch_to_w(model):
    a = model.a.detach().numpy()
    b = model.b.detach().numpy()
    w = model.w.detach().numpy()
    
    W = list(a) + list(b) + list(w.reshape(nv * nh))
    
    return np.array(W)

In [5]:
states = np.array([[1,0,0],[1,1,0],[1,0,1],[1,1,1]])
idx = np.random.choice(np.arange(len(states)), N)
data = states[idx]
data = torch.from_numpy(data)
train_loader = torch.utils.data.DataLoader(dataset=data.to(torch.float),
                                           batch_size=batch_size,
                                           shuffle=True)

In [6]:
def train(rbm, train_loader, learning_rate, k, training_epochs):
    optimizer = torch.optim.Adam(rbm.parameters(), lr=learning_rate)

    for epoch in range(training_epochs):
        
        epoch_cost = 0.
        
        for batch in train_loader:
            
            batch = batch.view(-1, nv)
            
            v = rbm.forward(batch)

            cost = torch.mean(rbm.free_energy(batch)) - torch.mean(rbm.free_energy(v))
            
            epoch_cost += cost.item()
            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

        print('Epoch [{}/{}], cost: {:.4f}'.format(epoch+1, training_epochs, epoch_cost))

In [7]:
model = RBM(nv, nh)
train(model, train_loader, learning_rate, k, epochs)

Epoch [1/25], cost: -20.2902
Epoch [2/25], cost: -11.1853
Epoch [3/25], cost: -6.5335
Epoch [4/25], cost: -4.1606
Epoch [5/25], cost: -2.9695
Epoch [6/25], cost: -1.8236
Epoch [7/25], cost: -1.4269
Epoch [8/25], cost: -1.3068
Epoch [9/25], cost: -0.8032
Epoch [10/25], cost: -0.5879
Epoch [11/25], cost: -1.0201
Epoch [12/25], cost: -1.1388
Epoch [13/25], cost: -0.6922
Epoch [14/25], cost: -0.5790
Epoch [15/25], cost: -0.0776
Epoch [16/25], cost: -0.1667
Epoch [17/25], cost: -0.4511
Epoch [18/25], cost: -0.0281
Epoch [19/25], cost: -0.0869
Epoch [20/25], cost: -0.1665
Epoch [21/25], cost: -0.2613
Epoch [22/25], cost: -0.1955
Epoch [23/25], cost: -0.1999
Epoch [24/25], cost: -0.0606
Epoch [25/25], cost: 0.0096


In [10]:
W = torch_to_w(model)